In [2]:
import numpy as np
import cv2
from skimage import io
from skimage import transform
from skimage.transform import downscale_local_mean, resize
from sklearn.linear_model import Lasso
import os


# Path to the input video
input_video_path = "Testvideo (online-video-cutter.com).mp4"

# Check if the video file exists
if not os.path.isfile(input_video_path):
    print("Error: Input video file does not exist.")
    exit()

# Load the video
cap = cv2.VideoCapture(input_video_path)


width = 0
height = 0
n_pixels = 0  # Assuming frames are of the same size as video frames

# Check if the video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break   
    # Convert frame to grayscale
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Downscale the original image to a smaller size for faster processing
    downscaled_image = frame_gray
    # Flatten the downsampled frame into a vector
    frame_vector = downscaled_image.flatten()
    sizeOfFrameVector = len(frame_vector)
    count=1
    for i in range(2,30):
           if(sizeOfFrameVector%i==0):
               count=i
    n_pixels=int(sizeOfFrameVector/count )
    height,width=downscaled_image.shape
    break

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print("Video properties:")
print("FPS:", fps)
print("Frame count:", frame_count)
print("Width:", width)
print("Height:", height)

# Load the sensing matrix
n_measurements = 5000  # Number of measurements
A = np.random.randn(n_measurements, n_pixels)

print(n_pixels)



output_video_path = "Output.mp4"
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# Process each frame of the video
# Process each frame of the video
temp = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    

    # Downscale the original image to a smaller size for faster processing
    frame = blurred_image = cv2.GaussianBlur(frame_gray, (5, 5), 0)

    # Reconstruct frame using compressive sensing
    print("Reconstructing frame:", temp)


    # Flatten the downsampled frame into a vector
    frame_vector = frame.flatten()
    



   
    #deviding vector in other part
    factor= int(sizeOfFrameVector/count )
    print(count," ", factor)         
    buffer=0
    # Initialize vector_of_frame as a list of lists
    vector_of_frame = [[] for _ in range(count)]

    # Assign values to vector_of_frame
    buffer = 0
    for i in range(0, count):
        for j in range(0, factor):
            vector_of_frame[i].append(frame_vector[buffer])
            buffer += 1
         


    for i in range(0,count):
            vec1=vector_of_frame[i]
            # Take measurements
            y = np.dot(A, vec1)

            # Reconstruct the frame using L1 minimization (Lasso)
            lasso = Lasso(alpha=0.01, max_iter=10000)  # Adjust alpha and max_iter
            lasso.fit(A, y)
            vector_of_frame[i] = lasso.coef_

    # Reshape the reconstructed frame vector into the downsampled shape
    # Initialize reconstructed_frame_vector as an empty list
    reconstructed_frame_vector = []

    # Append values to reconstructed_frame_vector
    for i in range(0, count):
        for j in range(0, factor):
            reconstructed_frame_vector.append(vector_of_frame[i][j])

    # Reshape the reconstructed frame vector into the downsampled shape
    reconstructed_downscaled_frame = np.array(reconstructed_frame_vector).reshape(frame.shape)

    

    # Convert the upscaled reconstructed frame to uint8 mode
    upscaled_reconstructed_frame_uint8 = (reconstructed_downscaled_frame * 255).astype(np.uint8)
     
     
    reconstructed_frame = upscaled_reconstructed_frame_uint8


    # Write reconstructed frame to output video
    print("Writing frame to output video:", temp)
   
    
    # Save the upscaled reconstructed image
    output_filename = f"frame_{temp}.jpg"
    io.imsave(output_filename, reconstructed_frame) 
    # Write reconstructed frame to output video
    out.write(cv2.cvtColor(reconstructed_frame, cv2.COLOR_GRAY2BGR))
    temp += 1
    

# Release video capture and writer objects
cv2.destroyAllWindows() 
out.release()
cap.release()





Video properties:
FPS: 29.7
Frame count: 744
Width: 106
Height: 190
1007
Reconstructing frame: 0
20   1007


ValueError: Complex data not supported
[  7726.46607809-535.89642716j -36617.37582047+921.91373074j
 -52517.65484624-214.17991472j ... -11618.70913071-851.8317396j
  45980.76264328+179.14825763j  56188.83518192+963.35016074j]
